In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Завантажуємо датасет `feature_data.csv`

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Load data from CSV file
df = pd.read_csv('/content/drive/MyDrive/ARCHIVE/Archive/Курсові/4 курс 1 семестр/Датасет/feature_data.csv')
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Split data into features (X) and labels (y)
X = df.drop('label', axis=1)
y = df['label']

class_names = ['DJI', 'FutabaT14', 'FutabaT7', 'Graupner', 'Noise', 'Taranis', 'Turnigy']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

##Метод найближчих сусідів [K-Nearest Neighbors (KNN)]
####Визначення оптимальних параметрів:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

param_grid = {
    'n_neighbors': [3, 5, 7, 10],
    'weights': ['uniform', 'distance'],
    'algorithm': ['ball_tree', 'kd_tree', 'brute'],
}

knn = KNeighborsClassifier()

# Initialize the GridSearchCV object
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Get the best parameters and use them to predict
print("Best parameters found: ", grid_search.best_params_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
Best parameters found:  {'algorithm': 'ball_tree', 'n_neighbors': 10, 'weights': 'distance'}


###Оцінювання найкращої KNN моделі

In [ ]:
best_knn_model = grid_search.best_estimator_
y_pred = best_knn_model.predict(X_val)
print(classification_report(y_val, y_pred, target_names=class_names))

              precision    recall  f1-score   support

         DJI       0.45      0.11      0.17       426
   FutabaT14       0.59      0.47      0.52      1397
    FutabaT7       0.43      0.16      0.23       751
    Graupner       0.63      0.46      0.53      1281
       Noise       0.83      0.95      0.88     10448
     Taranis       0.79      0.87      0.83      3289
     Turnigy       0.58      0.50      0.53      2149

    accuracy                           0.77     19741
   macro avg       0.61      0.50      0.53     19741
weighted avg       0.74      0.77      0.75     19741



###Збереження моделі на диск

In [ ]:
import joblib

knn_model_path = '/content/drive/MyDrive/ARCHIVE/Archive/Курсові/4 курс 1 семестр/Датасет/knn_model.joblib'
joblib.dump(best_knn_model, knn_model_path)

['/content/drive/MyDrive/ARCHIVE/Archive/Курсові/4 курс 1 семестр/Датасет/knn_model.joblib']

##XGBoost
####Визначення оптимальних параметрів:

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [6, 10, 15],
}

# Initialize an XGBoost classifier object
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', verbosity=2)
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Get the best parameters
best_parameters = grid_search.best_params_
print(f"Best parameters: {best_parameters}")

Fitting 3 folds for each of 9 candidates, totalling 27 fits
Best parameters: {'max_depth': 10, 'n_estimators': 200}


####Отримуємо найкращю XGBoost модель та оцінюємо її

In [ ]:
best_xgboost_model = grid_search.best_estimator_
y_pred = best_xgboost_model.predict(X_val)

print(classification_report(y_val, y_pred, target_names=class_names))

              precision    recall  f1-score   support

         DJI       0.87      0.60      0.71       426
   FutabaT14       0.76      0.76      0.76      1397
    FutabaT7       0.78      0.57      0.66       751
    Graupner       0.83      0.79      0.81      1281
       Noise       1.00      1.00      1.00     10448
     Taranis       0.93      0.95      0.94      3289
     Turnigy       0.74      0.85      0.79      2149

    accuracy                           0.92     19741
   macro avg       0.84      0.79      0.81     19741
weighted avg       0.92      0.92      0.92     19741



####Зберігаємо на диск

In [ ]:
import joblib

xgb_model_path = '/content/drive/MyDrive/ARCHIVE/Archive/Курсові/4 курс 1 семестр/Датасет/xgboost_model.joblib'
joblib.dump(best_xgboost_model, xgb_model_path)

['/content/drive/MyDrive/ARCHIVE/Archive/Курсові/4 курс 1 семестр/Датасет/xgboost_model.joblib']